In [ ]:
# create spark session
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("Introduction to Spark")
    .master("local[*]")
    .getOrCreate()
)
spark

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/07/15 10:54:01 WARN Utils: Your hostname, DESKTOP-DMC84B8, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/07/15 10:54:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/15 10:54:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 38254)
Traceback (most recent call last):
  File "/home/zap/miniconda3/envs/spark-env/lib/python3.12/socketserver.py", line 318, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/home/zap/miniconda3/envs/spark-env/lib/python3.12/socketserver.py", line 349, in process_request
    self.finish_request(request, client_address)
  File "/home/zap/miniconda3/envs/spark-env/lib/python3.12/socketserver.py", line 362, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/home/zap/miniconda3/envs/spark-env/lib/python3.12/socketserver.py", line 766, in __init__
    self.handle()
  File "/home/zap/miniconda3/envs/spark-env/lib/python3.12/site-packages/pyspark/accumulators.py", line 299, in handle
    poll(accum_updates)
  File "/home/zap/miniconda3/envs/spark-env/lib/python3.12/site-packages/pyspark/accumulators.py", line 27

In [25]:
# create sample data
emp_data = [
    ["001","101","John Doe","30","Male","50000","2015-01-01"],
    ["002","101","Jane Smith","25","Female","45000","2016-02-15"],
    ["003","102","Bob Brown","35","Male","55000","2014-05-01"],
    ["004","102","Alice Lee","28","Female","48000","2017-09-30"],
    ["005","103","Jack Chan","40","Male","60000","2013-04-01"],
    ["006","103","Jill Wong","32","Female","52000","2018-07-01"],
    ["007","101","James Johnson","42","Male","70000","2012-03-15"],
    ["008","102","Kate Kim","29","Female","51000","2019-10-01"],
    ["009","103","Tom Tan","33","Male","58000","2016-06-01"],
    ["010","104","Lisa Lee","27","Female","47000","2018-08-01"],
    ["011","104","David Park","38","Male","65000","2015-11-01"],
    ["012","105","Susan Chen","31","Female","54000","2017-02-15"],
    ["013","106","Brian Kim","45","Binary","75000","2011-07-01"],
    ["014","107","Emily Lee","26", None,"46000","2019-01-01"],
    ["015","106","Michael Lee","37","Male","63000","2014-09-30"],
    ["016","107","Kelly Zhang","30","Female","49000","2018-04-01"],
    ["017","105","George Wang","34","Male","57000","2016-03-15"],
    ["018","104","Nancy Liu","29","Mechanic","50000","2017-06-01"],
    ["019","103","Steven Chen","36","Male","62000","2015-08-01"],
    ["020","102","Grace Kim","32", None,"53000","2018-11-01"]
]

emp_schema = "employee_id string, department_id string, name string, age string, gender string, salary string, hire_date string"

In [26]:
emp = spark.createDataFrame(data=emp_data, schema=emp_schema)
emp.show()

+-----------+-------------+-------------+---+--------+------+----------+
|employee_id|department_id|         name|age|  gender|salary| hire_date|
+-----------+-------------+-------------+---+--------+------+----------+
|        001|          101|     John Doe| 30|    Male| 50000|2015-01-01|
|        002|          101|   Jane Smith| 25|  Female| 45000|2016-02-15|
|        003|          102|    Bob Brown| 35|    Male| 55000|2014-05-01|
|        004|          102|    Alice Lee| 28|  Female| 48000|2017-09-30|
|        005|          103|    Jack Chan| 40|    Male| 60000|2013-04-01|
|        006|          103|    Jill Wong| 32|  Female| 52000|2018-07-01|
|        007|          101|James Johnson| 42|    Male| 70000|2012-03-15|
|        008|          102|     Kate Kim| 29|  Female| 51000|2019-10-01|
|        009|          103|      Tom Tan| 33|    Male| 58000|2016-06-01|
|        010|          104|     Lisa Lee| 27|  Female| 47000|2018-08-01|
|        011|          104|   David Park| 38|    Ma

In [27]:
# case when
# apply on a gender attribute: 'M' if male, 'F' if female and null otherwise
from pyspark.sql.functions import when, col

emp_gender_fixed = emp.withColumn("gender_fixed", when(col("gender") == "Male", "M")
                                  .when(col("gender") == "Female", "F")
                                  .otherwise(None))
emp_gender_fixed.show()

+-----------+-------------+-------------+---+--------+------+----------+------------+
|employee_id|department_id|         name|age|  gender|salary| hire_date|gender_fixed|
+-----------+-------------+-------------+---+--------+------+----------+------------+
|        001|          101|     John Doe| 30|    Male| 50000|2015-01-01|           M|
|        002|          101|   Jane Smith| 25|  Female| 45000|2016-02-15|           F|
|        003|          102|    Bob Brown| 35|    Male| 55000|2014-05-01|           M|
|        004|          102|    Alice Lee| 28|  Female| 48000|2017-09-30|           F|
|        005|          103|    Jack Chan| 40|    Male| 60000|2013-04-01|           M|
|        006|          103|    Jill Wong| 32|  Female| 52000|2018-07-01|           F|
|        007|          101|James Johnson| 42|    Male| 70000|2012-03-15|           M|
|        008|          102|     Kate Kim| 29|  Female| 51000|2019-10-01|           F|
|        009|          103|      Tom Tan| 33|    Male|

In [28]:
## antherway using expr
from pyspark.sql.functions import expr

emp_gender_fixed_1 = emp.withColumn(
    "gender_fixed",
    expr(
        """CASE
            WHEN gender = 'Male' THEN 'M'
            WHEN gender = 'Female' THEN 'F'
            ELSE null
            END
        """)
)
emp_gender_fixed_1.show()

+-----------+-------------+-------------+---+--------+------+----------+------------+
|employee_id|department_id|         name|age|  gender|salary| hire_date|gender_fixed|
+-----------+-------------+-------------+---+--------+------+----------+------------+
|        001|          101|     John Doe| 30|    Male| 50000|2015-01-01|           M|
|        002|          101|   Jane Smith| 25|  Female| 45000|2016-02-15|           F|
|        003|          102|    Bob Brown| 35|    Male| 55000|2014-05-01|           M|
|        004|          102|    Alice Lee| 28|  Female| 48000|2017-09-30|           F|
|        005|          103|    Jack Chan| 40|    Male| 60000|2013-04-01|           M|
|        006|          103|    Jill Wong| 32|  Female| 52000|2018-07-01|           F|
|        007|          101|James Johnson| 42|    Male| 70000|2012-03-15|           M|
|        008|          102|     Kate Kim| 29|  Female| 51000|2019-10-01|           F|
|        009|          103|      Tom Tan| 33|    Male|

In [13]:
# replace in string
## replace J to Z in name
from pyspark.sql.functions import regexp_replace, col

emp_name_fixed = emp.withColumn("new_name", regexp_replace(col("name"), "J", "Z"))
emp_name_fixed.show()

+-----------+-------------+-------------+---+--------+------+----------+-------------+
|employee_id|department_id|         name|age|  gender|salary| hire_date|     new_name|
+-----------+-------------+-------------+---+--------+------+----------+-------------+
|        001|          101|     John Doe| 30|    Male| 50000|2015-01-01|     Zohn Doe|
|        002|          101|   Jane Smith| 25|  Female| 45000|2016-02-15|   Zane Smith|
|        003|          102|    Bob Brown| 35|    Male| 55000|2014-05-01|    Bob Brown|
|        004|          102|    Alice Lee| 28|  Female| 48000|2017-09-30|    Alice Lee|
|        005|          103|    Jack Chan| 40|    Male| 60000|2013-04-01|    Zack Chan|
|        006|          103|    Jill Wong| 32|  Female| 52000|2018-07-01|    Zill Wong|
|        007|          101|James Johnson| 42|    Male| 70000|2012-03-15|Zames Zohnson|
|        008|          102|     Kate Kim| 29|  Female| 51000|2019-10-01|     Kate Kim|
|        009|          103|      Tom Tan| 3

In [21]:
# convert date of emp_name_fixed
from pyspark.sql.functions import to_date

emp_date_fixed = emp_name_fixed.withColumn("hire_date", to_date(col("hire_date"), format="yyyy-MM-dd"))
emp_date_fixed.show()

+-----------+-------------+-------------+---+--------+------+----------+-------------+
|employee_id|department_id|         name|age|  gender|salary| hire_date|     new_name|
+-----------+-------------+-------------+---+--------+------+----------+-------------+
|        001|          101|     John Doe| 30|    Male| 50000|2015-01-01|     Zohn Doe|
|        002|          101|   Jane Smith| 25|  Female| 45000|2016-02-15|   Zane Smith|
|        003|          102|    Bob Brown| 35|    Male| 55000|2014-05-01|    Bob Brown|
|        004|          102|    Alice Lee| 28|  Female| 48000|2017-09-30|    Alice Lee|
|        005|          103|    Jack Chan| 40|    Male| 60000|2013-04-01|    Zack Chan|
|        006|          103|    Jill Wong| 32|  Female| 52000|2018-07-01|    Zill Wong|
|        007|          101|James Johnson| 42|    Male| 70000|2012-03-15|Zames Zohnson|
|        008|          102|     Kate Kim| 29|  Female| 51000|2019-10-01|     Kate Kim|
|        009|          103|      Tom Tan| 3

In [23]:
# add date column
## add current date, current timestamp
from pyspark.sql.functions import current_date, current_timestamp

emp_current = emp_date_fixed.withColumns({
    "current_date": current_date(),
    "current_timestamp": current_timestamp()
})
emp_current.show(truncate=False)

+-----------+-------------+-------------+---+--------+------+----------+-------------+------------+--------------------------+
|employee_id|department_id|name         |age|gender  |salary|hire_date |new_name     |current_date|current_timestamp         |
+-----------+-------------+-------------+---+--------+------+----------+-------------+------------+--------------------------+
|001        |101          |John Doe     |30 |Male    |50000 |2015-01-01|Zohn Doe     |2025-07-15  |2025-07-15 11:13:35.310868|
|002        |101          |Jane Smith   |25 |Female  |45000 |2016-02-15|Zane Smith   |2025-07-15  |2025-07-15 11:13:35.310868|
|003        |102          |Bob Brown    |35 |Male    |55000 |2014-05-01|Bob Brown    |2025-07-15  |2025-07-15 11:13:35.310868|
|004        |102          |Alice Lee    |28 |Female  |48000 |2017-09-30|Alice Lee    |2025-07-15  |2025-07-15 11:13:35.310868|
|005        |103          |Jack Chan    |40 |Male    |60000 |2013-04-01|Zack Chan    |2025-07-15  |2025-07-15 1

In [31]:
# drop null gender records
emp_dropna = emp.na.drop()
emp_dropna.show()

+-----------+-------------+-------------+---+--------+------+----------+
|employee_id|department_id|         name|age|  gender|salary| hire_date|
+-----------+-------------+-------------+---+--------+------+----------+
|        001|          101|     John Doe| 30|    Male| 50000|2015-01-01|
|        002|          101|   Jane Smith| 25|  Female| 45000|2016-02-15|
|        003|          102|    Bob Brown| 35|    Male| 55000|2014-05-01|
|        004|          102|    Alice Lee| 28|  Female| 48000|2017-09-30|
|        005|          103|    Jack Chan| 40|    Male| 60000|2013-04-01|
|        006|          103|    Jill Wong| 32|  Female| 52000|2018-07-01|
|        007|          101|James Johnson| 42|    Male| 70000|2012-03-15|
|        008|          102|     Kate Kim| 29|  Female| 51000|2019-10-01|
|        009|          103|      Tom Tan| 33|    Male| 58000|2016-06-01|
|        010|          104|     Lisa Lee| 27|  Female| 47000|2018-08-01|
|        011|          104|   David Park| 38|    Ma

In [ ]:
# fill null values
emp_nafilled = emp.na.fill("Unknown")
emp_nafilled.show()

+-----------+-------------+-------------+---+--------+------+----------+
|employee_id|department_id|         name|age|  gender|salary| hire_date|
+-----------+-------------+-------------+---+--------+------+----------+
|        001|          101|     John Doe| 30|    Male| 50000|2015-01-01|
|        002|          101|   Jane Smith| 25|  Female| 45000|2016-02-15|
|        003|          102|    Bob Brown| 35|    Male| 55000|2014-05-01|
|        004|          102|    Alice Lee| 28|  Female| 48000|2017-09-30|
|        005|          103|    Jack Chan| 40|    Male| 60000|2013-04-01|
|        006|          103|    Jill Wong| 32|  Female| 52000|2018-07-01|
|        007|          101|James Johnson| 42|    Male| 70000|2012-03-15|
|        008|          102|     Kate Kim| 29|  Female| 51000|2019-10-01|
|        009|          103|      Tom Tan| 33|    Male| 58000|2016-06-01|
|        010|          104|     Lisa Lee| 27|  Female| 47000|2018-08-01|
|        011|          104|   David Park| 38|    Ma

25/07/15 20:54:01 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 4924275 ms exceeds timeout 120000 ms
25/07/15 20:54:01 WARN SparkContext: Killing executors is not supported by current scheduler.
25/07/15 20:54:03 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:53)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:342)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:132)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$